In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns 

In [ ]:
Df = pd.read_csv("/kaggle/input/indian-startup-funding/startup_funding.csv", encoding='utf-8')

**This is my first kaggle Kernel so please give suggestions for improvements wherever required**
> Do upvote if you like the kernel

**Staring with looking at the data**

In [ ]:
Df.head()

In [ ]:
Df.tail()

 *Two column names are not correct, replacing the column name so that it will be easier to work later*

In [ ]:
Names_to_be_changed = {'Investorsxe2x80x99 Name':'Investor Name','InvestmentnType': 'Investment Type'}

In [ ]:
Df.rename(columns=Names_to_be_changed,inplace=True)

**Checking for indexes in Date column which cannot be converted into datetime object **

In [ ]:
invalid_index = []
for i in range(len(Df['Date ddmmyyyy'])):
    try:
        #print(i)
        pd.to_datetime(Df['Date ddmmyyyy'][i],format='%d/%m/%Y')
    except:
        print(i)
        invalid_index.append(i)

In [ ]:
Df['Date ddmmyyyy'][invalid_index]

**Relacing incorrect date with the correct ones and also creating Year,Month and Quarter columns from the same**

In [ ]:
Df['Date ddmmyyyy'][invalid_index]
Df['Date ddmmyyyy'][2570]
Correct_dates = ['05/07/2018','01/07/2015','10/07/2015','12/05/2015','12/05/2015','13/04/2015','15/01/2015','22/01/2015']
Df['Date ddmmyyyy'].iloc[invalid_index] = Correct_dates
Df['Date ddmmyyyy'] = pd.to_datetime(Df['Date ddmmyyyy'],format='%d/%m/%Y')
Df['Year'] = [x.year for x in Df['Date ddmmyyyy']]
Df['Month'] = [x.month for x in Df['Date ddmmyyyy']]
Df['Day']= [x.day_name() for x in Df['Date ddmmyyyy']]
Df['Quarter'] = [x.quarter for x in Df['Date ddmmyyyy']]

**In order to convert amount in numeric terms we need to remove **,**  and   **+** and replace undisclosed by zero**

In [ ]:
Df['Amount in USD'] = Df['Amount in USD'].apply(lambda x: str(x).replace(',','').replace('\\\\xc2\\\\xa0','').replace('+',''))
Df['Amount in USD'] = Df['Amount in USD'].apply(lambda x: x.replace('nan','undisclosed').replace('N/A','undisclosed')).replace('Undisclosed','undisclosed')

In [ ]:
Df['undisclosed'] = 0
Df.loc[Df['Amount in USD'] == 'undisclosed',['undisclosed']] = 1
Df.loc[Df['Amount in USD'] == 'undisclosed',['Amount in USD']] = 0
Df['Amount in USD'] = pd.to_numeric(Df['Amount in USD'])

**Number of deals done by year**

In [ ]:
plt.figure()
Df.groupby('Year')['Amount in USD'].agg(['count']).plot(kind = 'bar',figsize = (12,8))
plt.xlabel('Year')
plt.ylabel('Number of deals')

**Total Funding by Year**

In [ ]:
plt.figure()
Df.groupby('Year')['Amount in USD'].agg(['sum']).plot(kind = 'bar',figsize = (12,8))
plt.xlabel('Year')
plt.ylabel('Total Funding')

 ** Largest Amount of Funding was done in 2017. Coincidentally Indian stock markets were also aprreciating rapidly in 2017**

**Hot Quarter For Funding**

In [ ]:
fig = plt.figure(figsize = (12,8))
ax = fig.add_subplot(1,1,1)
Funding_by_month = Df.groupby(['Year','Quarter'])['Amount in USD'].agg(['sum']).reset_index()
sns.barplot(x = 'Quarter',y = 'sum',hue = 'Year',data = Funding_by_month,linewidth=2.5)
ax.set_xlabel("Quarter")
ax.set_ylabel("Sum")

**Quarter 3 seems be best time for funding**

**Let's dive into some text data**

There are mutiple special characters we need to clean them

In [ ]:
import re
def clean_names(some_text):
    some_text = str(some_text)
    some_text = some_text.replace('\\\\n',' ').replace('\\\\xe2\\\\x80\\\\x99',' ').replace('\\\\xc3\\\\x98',' ').replace('\\\\n\\\\n','').replace("\\\\xc2\\\\xa0",'')
    return some_text

**Hot Industry**

In [ ]:
Df['Industry Vertical'] = Df['Industry Vertical'].apply(lambda x : clean_names(x))

In [ ]:
Df['Industry Vertical'].replace(['eCommerce','ECommerce','Ecommerce','E-commerce'],'E-Commerce',inplace = True)

In [ ]:
Df['Industry Vertical'].value_counts().drop(labels = 'nan').sort_values(ascending=False)[0:10].plot(kind='bar',figsize = (12,8))

**Consumer Internet seems to be hottest sector with most number of startup funded in that industry. startups in sectors like E-Commerce and Healthcare are also getting funded**

In [ ]:

for name,group in Df.groupby('Year'): 
    print(name)
    try:
        plt.figure(figsize = (12,8))
        group['Industry Vertical'].value_counts().drop(labels = 'nan').sort_values(ascending=False)[0:10].plot(kind='bar')
        plt.title("Top 10 Industry")
        plt.xlabel("Startup funded in Year {}".format(name))
    except:
        plt.figure(figsize = (12,8))
        group['Industry Vertical'].value_counts().sort_values(ascending=False)[0:10].plot(kind='bar')
        plt.title("Top 10 Industry")
        plt.xlabel("Startup funded in Year {}".format(name))

If we see yearwise funding pattern E-Commerce is leading the way in 2019, Cosumer Internet which was at Top in previous year is not in TOP 10.

**Top Investors**

In the Investors name column wherever one startup is funded by multiple people/organisation they are all in a string joined by (, & and) we need to get individual names from the string

In [ ]:
Df['Investor Name'] = Df['Investor Name'].apply(lambda x: clean_names(x))

In [ ]:
def get_individual_names(some_string):
    some_string = some_string.split('and')
    some_string = ','.join(some_string)
    some_string = some_string.split('&')
    some_string = ','.join(some_string)
    some_string = some_string.split(',')
    modified_string_list = []
    for string in some_string:
        string = string.strip()
        modified_string_list.append(string)
    return modified_string_list

In [ ]:
Investor_names = Df['Investor Name'].apply(lambda x: get_individual_names(x))

**Remove some names which are common but are not so useful**

In [ ]:
names_to_be_removed = ['',' ','an','An','S','Undisclosed Investors','Undisclosed investors','undisclosed investors','Undisclosed','undisclosed','others','Others','nan']

In [ ]:
Investor_names_list = []
for investors in Investor_names:
    for investor in investors:
        if(investor not in names_to_be_removed):
            Investor_names_list.append(investor)
   

In [ ]:
from collections import Counter
Investors = pd.Series(Counter(Investor_names_list))
Investors.sort_values(ascending = False)[0:10].plot(kind = 'bar',figsize = (12,8))

**Most Active investment firms / Individuals**

**Startup funded in most number of rounds**

In [ ]:
Df['Startup Name'].value_counts().sort_values(ascending=False)[0:10].plot(kind='bar',figsize= (12,8))

**Which city has the best startup culture OR where the startups are most funded**

In [ ]:
Df['City  Location'] = Df['City  Location'].apply(lambda x: clean_names(x))

In [ ]:
Df['City  Location'].replace('Gurgaon','Gurugram',inplace = True)
Df['City  Location'].replace('Bengaluru','Bangalore',inplace = True)

In [ ]:
Df['City  Location'].value_counts().sort_values(ascending=False).drop('nan')[0:10].plot(kind='bar',figsize = (12,8))

**Funding by Investment type**

In [ ]:
Df['Investment Type'] = Df['Investment Type'].apply(lambda x : clean_names(x))

In [ ]:
Df['Investment Type'].value_counts().sort_values(ascending = False)[0:10].plot(kind = 'bar',figsize = (12,8))

**Amount of funding by Investment type**

In [ ]:
fig = plt.figure(figsize = (12,8))
Funding_by_type = Df.groupby(['Investment Type'])['Amount in USD'].agg(['sum']).sort_values(by = 'sum',ascending = False).reset_index()[0:10]
sns.barplot(x = 'Investment Type',y = 'sum',data = Funding_by_type,linewidth=2.5)
plt.xticks(rotation = 90)
plt.ylabel('Total investment')